In [3]:
import io
import zipfile
import requests
import networkx as nx
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline


  

import networkx as nx
import pickle

import random
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [4]:
url = ('https://github.com/ipython-books/'
       'cookbook-2nd-data/blob/master/'
       'road.zip?raw=true')
r = io.BytesIO(requests.get(url).content)
zipfile.ZipFile(r).extractall('data')
g = nx.read_shp('data/tl_2013_06_prisecroads.shp')

In [5]:
sgs = [g.subgraph(c) for c in nx.connected_components(g.to_undirected())]
i = np.argmax([len(sg) for sg in sgs])
sg = sgs[i]
len(sg)

464

In [6]:
def get_path(n0, n1):
    """If n0 and n1 are connected nodes in the graph,
    this function returns an array of point
    coordinates along the road linking these two
    nodes."""
    return np.array(json.loads(sg[n0][n1]['Json'])
                    ['coordinates'])


In [7]:
# from [https://stackoverflow.com/a/8859667/1595060](https://stackoverflow.com/a/8859667/1595060)
EARTH_R = 6372.8

def geocalc(lat0, lon0, lat1, lon1):
    """Return the distance (in km) between two points
    in geographical coordinates."""
    lat0 = np.radians(lat0)
    lon0 = np.radians(lon0)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    dlon = lon0 - lon1
    y = np.sqrt((np.cos(lat1) * np.sin(dlon)) ** 2 +
        (np.cos(lat0) * np.sin(lat1) - np.sin(lat0) *
         np.cos(lat1) * np.cos(dlon)) ** 2)
    x = np.sin(lat0) * np.sin(lat1) + \
        np.cos(lat0) * np.cos(lat1) * np.cos(dlon)
    c = np.arctan2(y, x)
    return EARTH_R * c


In [8]:
def get_path_length(path):
    return np.sum(geocalc(path[1:, 1], path[1:, 0],
                          path[:-1, 1], path[:-1, 0]))

In [9]:
# Compute the length of the road segments.
for n0, n1 in sg.edges:
    path = get_path(n0, n1)
    distance = get_path_length(path)
    sg.edges[n0, n1]['distance'] = distance


In [81]:
pairs = []
#look for connected nodes
for ix, x in enumerate(list(sg.nodes)):
    for iy, y in enumerate(list(sg.nodes)):
        if nx.has_path(sg,x,y) and x != y:
            pairs.append((ix, iy))
len(pairs)

2204

In [82]:
pairs = []
sg_ud = sg.to_undirected()
#look for connected nodes
for ix, x in enumerate(list(sg_ud.nodes)):
    for iy, y in enumerate(list(sg_ud.nodes)):
        if nx.has_path(sg_ud,x,y) and x != y:
            pairs.append((ix, iy))
len(pairs)

214832

In [12]:
#
pos0 = list(sg.nodes)[194]
pos1 = list(sg.nodes)[372]

In [13]:
nodes = np.array(sg.nodes())
# Get the closest nodes in the graph.
pos0_i = np.argmin(
    np.sum((nodes[:, ::-1] - pos0)**2, axis=1))
pos1_i = np.argmin(
    np.sum((nodes[:, ::-1] - pos1)**2, axis=1))


In [14]:
# Compute the shortest path.
path = nx.shortest_path(
    sg,
    source=tuple(pos0),
    target=tuple(pos1),
    weight='distance') #distance is from edge data shown below
len(path)


2

In [15]:
#take a look at the edges data
# we can see that we have a distance object in the json that we are using as the "weight" above
#we therefore need to replace this distance with a metric that includes the other information we are interested in
[e for e in sg.edges.data()][0]

((-121.586642, 39.14501299999999),
 (-121.59112900000001, 39.14465899999999),
 {'LINEARID': '1104493254357',
  'FULLNAME': 'State Rte 70',
  'RTTYP': 'S',
  'MTFCC': 'S1200',
  'ShpName': 'tl_2013_06_prisecroads',
  'Wkb': b"\x00\x00\x00\x00\x02\x00\x00\x00\x16\xc0^e\x8b\x8a\xe3\x1dq@C\x92\x8f\xc96?V\xc0^e\x8d\xdaH\xb6R@C\x92\x8f\x10\xa9\x9bn\xc0^e\x94\xd0\xdc\xfc\xc6@C\x92\x8c\xefg+\x87\xc0^e\x97$tS\x8f@C\x92\x8c?>\x03q\xc0^e\x99\xaa`\x91:@C\x92\x8b\xda\x945\xac\xc0^e\xa1@W\x08%@C\x92\x8a\xbd]\xc3\xff\xc0^e\xa3\xcau\x03\xb9@C\x92\x8aa\x17r\x0c\xc0^e\xa82\xb9\x90\xb0@C\x92\x8aX\xb3\xf6;\xc0^e\xaa\xc1\tJ,@C\x92\x8a\x04\xd1 \x18\xc0^e\xad\xe6W\xb8N@C\x92\x89\xa0'RS\xc0^e\xaf\xcc\xe1\xc5\x82@C\x92\x89m\xd2kq\xc0^e\xb2R\xce\x03/@C\x92\x88\xcep:\xfa\xc0^e\xb4\xbf\x8f\xcdh@C\x92\x88?\xd5\x02$\xc0^e\xb9\xfd\xbd/\xa2@C\x92\x87]V\xf3,\xc0^e\xbc\x8c\x0c\xe9\x1d@C\x92\x86\xf0I\xa9\x96\xc0^e\xbf\x05c\xed\x10@C\x92\x86\x9cf\xd3s\xc0^e\xc6y\xcct\xb9@C\x92\x85\xa0\xbeQ\x07\xc0^e\xc8\xf3#x\xab@C\x92\x

In [16]:
roads = pd.DataFrame(
    [sg.edges[path[i], path[i + 1]]
     for i in range(len(path) - 1)],
    columns=['FULLNAME', 'MTFCC',
             'RTTYP', 'distance'])
roads


,FULLNAME,MTFCC,RTTYP,distance
0,Golden State Fwy,S1100,M,22.014648


In [17]:
roads['distance'].sum()


22.014647832843167

In [18]:
def get_full_path(path):
    """Return the positions along a path."""
    p_list = []
    curp = None
    for i in range(len(path) - 1):
        p = get_path(path[i], path[i + 1])
        if curp is None:
            curp = p
        if (np.sum((p[0] - curp) ** 2) >
                np.sum((p[-1] - curp) ** 2)):
            p = p[::-1, :]
        p_list.append(p)
        curp = p[-1]
    return np.vstack(p_list)

In [19]:
#now we have a full gps path to plot
get_full_path(path)


array([[-118.627451,   34.502135],
       [-118.627433,   34.501895],
       [-118.627403,   34.501575],
       [-118.627386,   34.501488],
       [-118.627269,   34.500942],
       [-118.627178,   34.500666],
       [-118.627105,   34.50046 ],
       [-118.627041,   34.500315],
       [-118.626905,   34.499981],
       [-118.626767,   34.499693],
       [-118.626663,   34.499475],
       [-118.626549,   34.499267],
       [-118.626172,   34.498677],
       [-118.62586 ,   34.498272],
       [-118.625288,   34.497552],
       [-118.623987,   34.495782],
       [-118.623439,   34.495035],
       [-118.621699,   34.492738],
       [-118.621183,   34.492016],
       [-118.620909,   34.491657],
       [-118.619342,   34.489591],
       [-118.619052,   34.489214],
       [-118.618709,   34.488763],
       [-118.617309,   34.486884],
       [-118.617153,   34.486674],
       [-118.616686,   34.486022],
       [-118.616309,   34.485471],
       [-118.615884,   34.484752],
       [-118.615633,

In [44]:
list(sg.nodes)[0]

(-121.586642, 39.14501299999999)

In [51]:
set(sg._graph.nodes) - set(sg.nodes)

{(-119.043108, 36.10421699999999),
 (-117.92936500000002, 33.62128899999999),
 (-118.19732300000001, 36.79873599999999),
 (-119.620657, 36.866386),
 (-122.428109, 39.79993799999999),
 (-122.500221, 37.71895599999999),
 (-117.016772, 34.89869999999999),
 (-122.475644, 37.743052),
 (-117.506735, 34.065709999999996),
 (-117.478387, 33.897756),
 (-117.274641, 33.94177599999999),
 (-123.098615, 38.977263),
 (-120.72035000000001, 38.312677),
 (-119.286655, 36.518968),
 (-122.017291, 39.457316999999996),
 (-122.927914, 38.323821999999986),
 (-118.006086, 35.85989699999999),
 (-119.749354, 36.75772899999999),
 (-120.36166700000001, 36.732755999999995),
 (-116.949946, 32.567529),
 (-119.274872, 35.891885999999985),
 (-119.317518, 37.43011299999999),
 (-118.568381, 34.171876999999995),
 (-119.331372, 36.152754),
 (-117.43389, 34.01178999999999),
 (-122.20237900000001, 39.749411999999985),
 (-119.374844, 37.50399699999999),
 (-122.422135, 37.789412),
 (-120.948801, 38.364954999999995),
 (-120.033

In [52]:
set(sg.nodes) - set(sg._graph.nodes)

set()

In [40]:
G = sg._graph
nx.get_node_attributes(G, 'pos')
edge_trace = Scatter(
x=[],
y=[],
line=Line(width=0.5,color='#888'),
hoverinfo='none',
mode='lines')

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]




ValueError: not enough values to unpack (expected 2, got 0)

In [55]:
node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=False,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='Hot',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
line=dict(width=2)))

In [39]:
G.nodes[edge[0]]

{}

In [33]:
G.nodes

NodeView(((-118.395179, 37.361458999999996), (-118.395325, 37.37579699999999), (-118.464921, 37.376373), (-118.395425, 37.37393), (-118.91436000000002, 35.281551), (-118.91439100000001, 35.266982999999996), (-117.669929, 35.629847999999996), (-117.67002, 35.62245), (-117.66993000000001, 35.651578), (-117.909608, 34.149315), (-117.90740900000002, 34.14509499999999), (-118.30982, 33.743397), (-118.309635, 33.744631999999996), (-118.30969400000001, 33.743367), (-118.260973, 33.749789), (-118.25459000000001, 33.752953), (-118.338596, 34.09071), (-118.337071, 34.108233), (-118.33618, 34.109612999999996), (-118.33633900000001, 34.11450599999999), (-118.606055, 34.215564), (-118.60605400000001, 34.257227), (-118.385794, 33.842055), (-118.395963, 33.877100999999996), (-114.57091100000001, 33.618059999999986), (-114.572813, 33.701554), (-116.972072, 33.8117), (-116.971517, 33.794401), (-120.101684, 39.27541999999999), (-120.152229, 39.316290999999985), (-121.486546, 38.589407), (-121.4784459999

In [79]:
sg.to_undirected()

In [20]:

def show_map(M, start=None, goal=None, path=None):
    G = M._graph
    pos = nx.get_node_attributes(G, 'pos')
    edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    node_trace = Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=Marker(
            showscale=False,
            # colorscale options
            # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
            # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
            colorscale='Hot',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))
    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['x'].append(x)
        node_trace['y'].append(y)

    for node, adjacencies in enumerate(G.adjacency_list()):
        color = 0
        if path and node in path:
            color = 2
        if node == start:
            color = 3
        elif node == goal:
            color = 1
        # node_trace['marker']['color'].append(len(adjacencies))
        node_trace['marker']['color'].append(color)
        node_info = "Intersection " + str(node)
        node_trace['text'].append(node_info)

    fig = Figure(data=Data([edge_trace, node_trace]),
                 layout=Layout(
                    title='<br>Network graph made with Python',
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                   
                    xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

    iplot(fig)

In [23]:
show_map(sg, start=pos0, goal=pos1, path=path)

/home/owen/miniconda3/envs/smartcities3/lib/python3.8/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




AttributeError: 'DiGraph' object has no attribute 'node'

In [ ]:
n